### Multiple Producers and Consumer with Semaphores (Java) [4 points]

#### Producers and Consumers

Here is a producer/consumer problem with semaphores in Java. The producer produces the first `numIters` numbers; the consumer expects `numIters` numbers and prints their sum. The shared variable `data` is declared as `volatile`. This tells the compiler that the variable can change at any time and has to be read from memory. Without that, in the function `Consumer`, the compiler may assume that `data` does not change in the function and keep it in a register.

In [1]:
%%writefile ProducerConsumer.java
import java.util.concurrent.Semaphore;
import java.lang.String;
public class ProducerConsumer {
    public static Semaphore empty = new Semaphore(1);
    public static Semaphore full = new Semaphore(0);
    public static volatile int data; // shared buffer
    public static int numIters;
    static Thread makeproducer() {
        return new Thread() {
            public void run () {
                System.out.println("Producer created");
                for (int produced = 0; produced < numIters; produced++) {
                    try {empty.acquire();} catch (Exception e) {}
                    data = produced;
                    full.release();
                }
            }
        };
    }
    static Thread makeconsumer() {
        return new Thread() {
            public void run () {
                System.out.println("Consumer created");
                int sum = 0;
                for (int consumed = 0; consumed < numIters; consumed++) {
                    try {full.acquire();} catch (Exception e) {}
                    sum += data;
                    empty.release();
                }
                System.out.println("For " + numIters + " iterations, the sum is " + sum);  
            }
        };
    }
    public static void main(String args[]) {
        numIters = Integer.parseInt(args[0]);
        Thread p = makeproducer(), c = makeconsumer();
       
        p.start(); c.start();
        try {p.join(); c.join();} catch(Exception e) {}
    }
}

Overwriting ProducerConsumer.java


In [2]:
!javac ProducerConsumer.java
!java ProducerConsumer 10

Producer created
Consumer created
For 10 iterations, the sum is 45


#### Multiple Producers and Consumers

Generalize the above Java implementation for an arbitrary number of producers/consumers. The program should take two parameters from the command line: the number of iterations of each producer/consumer and the number of producers/consumers. All producers and all consumers have the same number of iterations, and there are as many consumers as there are producers. The output should be modified such that each producer and consumer identifies itself when printing that they were created and when printing their sum. The main program should additionally print the expected total sum by computing that, not by collecting the sums of each consumer: the sum each consumer computes may be different now due to nondeterminism, but their total sum is unique. Recall that the sum of the first `n - 1` numbers is `n × (n - 1) / 2`.  For example
```bash
!java MultiProducerConsumer 10 3
```
may result in:
```
Consumer 0 created
Consumer 2 created
Producer 1 created
Producer 0 created
Producer 2 created
Consumer 1 created
For 10 iterations, the sum of consumer 2 is 36
For 10 iterations, the sum of consumer 1 is 45
For 10 iterations, the sum of consumer 0 is 54
The expected total sum is 135
```

In [3]:
%%writefile MultiProducerConsumer.java
import java.util.concurrent.Semaphore;
import java.lang.String;
public class MultiProducerConsumer {
    public static Semaphore empty = new Semaphore(1);
    public static Semaphore full = new Semaphore(0);
    public static volatile int data; // shared buffer
    public static int numIters;
    public static int numThreads;

    static Thread makeproducer(int id) {
        return new Thread() {
            public void run () {
                System.out.println("Producer " + id + " created");
                for (int produced = 0; produced < numIters; produced++) {
                    try {empty.acquire();} catch (Exception e) {}
                    data = produced;
                    full.release();
                }
            }
        };
    }
    static Thread makeconsumer(int id) {
        return new Thread() {
            public void run () {
                System.out.println("Consumer " + id + " created");
                int sum = 0;
                for (int consumed = 0; consumed < numIters; consumed++) {
                    try {full.acquire();} catch (Exception e) {}
                    sum += data;
                    empty.release();
                }
                System.out.println("For " + numIters + " iterations, the sum of consumer " + id + " is " + sum);  
            }
        };
    }
    public static void main(String args[]) {
        numIters = Integer.parseInt(args[0]);
        numThreads = Integer.parseInt(args[1]);
        Thread[] p = new Thread[numThreads];
        Thread[] c = new Thread[numThreads];
        for (int i = 0; i < numThreads; i ++) {
            p[i] = makeproducer(i);
            c[i] = makeconsumer(i); 
        }

        for (int i = 0; i < numThreads; i++){
            p[i].start();
            c[i].start();
        }

        for (int i = 0; i < numThreads; i++){
            try {
            p[i].join();
            c[i].join();
            } catch (Exception e) {}
        }

        System.out.println("The expected total sum is: " + (numIters * (numIters - 1) / 2) * numThreads);

    }
}

Overwriting MultiProducerConsumer.java


In [4]:
!javac MultiProducerConsumer.java

In [5]:
!java MultiProducerConsumer 10 3

Producer 0 created
Consumer 0 created
Producer 1 created
Consumer 1 created
For 10 iterations, the sum of consumer 0 is 45
For 10 iterations, the sum of consumer 1 is 45
Producer 2 created
Consumer 2 created
For 10 iterations, the sum of consumer 2 is 45
The expected total sum is: 135
